In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4599/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4599/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_40_8,0.992326,0.795267,0.904710,0.972368,0.004192,0.121539,0.081162,0.010599,0.029202,0.064747,0.984156,0.065767,76.949070,148.703850,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_18_7,0.992934,0.795190,0.907407,0.977262,0.003860,0.121584,0.078866,0.009732,0.029107,0.062127,0.985413,0.063105,77.114304,148.869084,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_40_4,0.992334,0.795040,0.904639,0.972499,0.004188,0.121673,0.081223,0.010549,0.029189,0.064712,0.984173,0.065731,76.951225,148.706005,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_41_2,0.992381,0.794895,0.904424,0.994460,0.004162,0.121759,0.081406,0.004427,0.029166,0.064511,0.984271,0.065527,76.963670,148.718450,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_41_6,0.992452,0.794305,0.904023,0.994221,0.004123,0.122110,0.081748,0.004618,0.029118,0.064214,0.984416,0.065225,76.982150,148.736930,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,model_3_42_0,0.986378,-0.691542,0.578365,0.892374,0.007441,1.004173,0.359124,0.024027,0.035056,0.086263,0.941878,0.087622,107.801413,214.346389,"Hidden Size=[12], regularizer=0.02, learning_r..."
623,model_3_48_1,0.993217,-0.702870,0.819397,0.986092,0.003705,1.010897,0.153827,0.006257,0.033798,0.060872,0.971058,0.061831,109.195937,215.740913,"Hidden Size=[12], regularizer=0.02, learning_r..."
624,model_3_48_5,0.994107,-0.748845,0.810702,0.985166,0.003219,1.038190,0.161233,0.006674,0.032946,0.056736,0.974857,0.057630,109.477355,216.022331,"Hidden Size=[12], regularizer=0.02, learning_r..."
625,model_3_47_8,0.993406,-0.879197,0.823293,0.989683,0.003602,1.115573,0.150509,0.005252,0.033156,0.060017,0.971865,0.060963,109.252481,215.797457,"Hidden Size=[12], regularizer=0.02, learning_r..."
